# Нелинейное программирование. Безусловная оптимизация

## Инициализация окружения

In [1]:
import numpy as np
from scipy.optimize import minimize, fmin_cg, fmin_bfgs, fmin_ncg
from scipy.misc import derivative
import csv

## Целевая функция

In [2]:
def target_function(x):
    x1 = x[0]
    x2 = x[1]
    return 17 * x1 ** 2 + 23 * x2 ** 2 - 8 * x1 * x2 - 182 * x1 - 266 * x2

## Вспомогательные функции для информативного вывода результатов

In [3]:
def write_to_csv_each_iteration(xk):
    writer.writerow((xk[0], xk[1], -target_function(xk)))
    print('x_1={x1}, x_2={x2}, f(X)={f}'.format(x1=xk[0], x2=xk[1], f=-target_function(xk)))

## Метод релаксации

## Метод Ньютона

In [4]:
X0 = np.array([0, 0])

file_name = '../report/data/ncg.csv'
f = open(file_name, 'w')
writer = csv.writer(f)
writer.writerow(('x1', 'x2', 'f'))
f.close()

## Метод сопряжённых градиентов (Conjugate Gradient method)

In [5]:
X0 = np.array([0, 0])

file_name = '../report/data/cg.csv'
f = open(file_name, 'w')
writer = csv.writer(f)
writer.writerow(('x1', 'x2', 'f'))
xopt, msg = fmin_cg(target_function, X0, callback=write_to_csv_each_iteration)
f.close()
print(xopt, msg)

x_1=5.361109094252702, x_2=7.835467138564297, f(X)=1495.3305834091711
x_1=7.125213729861292, x_2=7.185423921587326, f(X)=1567.1284201321148
x_1=7.054765530994868, x_2=6.994239380665157, f(X)=1567.9457254065308
x_1=7.000000326775502, x_2=6.999999844649399, f(X)=1567.9999999999973
Optimization terminated successfully.
         Current function value: -1568.000000
         Iterations: 4
         Function evaluations: 40
         Gradient evaluations: 10
7.00000032678 6.99999984465


## Метод Бройдена (BFGS)

In [6]:
X0 = np.array([0, 0])

file_name = '../report/data/bfgs.csv'
f = open(file_name, 'w')
writer = csv.writer(f)
writer.writerow(('x1', 'x2', 'f'))
xopt = fmin_bfgs(target_function, X0, callback=write_to_csv_each_iteration)
f.close()

print(xopt)

x_1=0.5703307547077343, x_2=0.8335603338898189, f(X)=307.8198499446669
x_1=3.1270769590102674, x_2=0.1430122836507337, f(X)=444.04017429337677
x_1=3.6011566481364348, x_2=0.982367647852593, f(X)=702.363923604765
x_1=4.625584461705584, x_2=2.7961118873142676, f(X)=1145.539251452151
x_1=6.999999284449173, x_2=6.9999989800202975, f(X)=1567.999999999973
x_1=7.000000464462481, x_2=7.000000180378583, f(X)=1567.9999999999964
x_1=6.999999889250844, x_2=7.000000160651894, f(X)=1567.9999999999989
Optimization terminated successfully.
         Current function value: -1568.000000
         Iterations: 7
         Function evaluations: 36
         Gradient evaluations: 9
[ 6.99999989  7.00000016]
